In [ ]:
! python3 setup.py install

In [7]:
from deltamic import Gaussian_psf, compute_spatial_frequency_grid,normalize_tensor,render_image_from_ftmesh,Fourier3dMesh,compute_box_size
from deltamic.mask_creation import create_instance_segmentation_mask,create_semantic_segmentation_mask
import trimesh
import numpy as np
import torch
import skimage.io as io 
from skimage import img_as_ubyte
from time import time
from dw3d import open_mesh_multitracker, write_mesh_bin, DCEL_Data

filename = "data/mesh_multimaterial.rec"
Verts,Faces_mm,_ = open_mesh_multitracker(filename)
box_shape = np.array([30,30,30])
box_size = compute_box_size(Verts,offset=0.1)
labels = create_instance_segmentation_mask(Verts,Faces_mm,box_shape,box_size)
contours = create_semantic_segmentation_mask(Verts,Faces_mm[:,:3],box_shape,box_size)

Number of remeshing iterations: 1
(25568, 3, 3)


In [ ]:
device = 'cpu'

Verts = torch.tensor(Verts,dtyadd_image=torch.float).to(device)
Faces_mm = torch.tensor(Faces_mm,dtype = torch.long).to(device)
Faces = Faces_mm[:,:3].contiguous()
print("Number of verts,faces",len(Verts),len(Faces_mm))
Faces_coeff = torch.zeros(len(Faces),dtype = torch.float, device = device)
for i,f in enumerate(Faces_mm.cpu().numpy()): 
    if f[3]==0 or f[4]==0:  
        Faces_coeff[i]=1
    else : 
        Faces_coeff[i]=2
        
print("Vertices shape: ",Verts.shape,"Faces shape: ", Faces.shape)
print("Min/Max x/y/z position of vertices: ",Verts.min(axis=0),Verts.max(axis=0))
print("box_size: ",box_size, "box_shape",box_shape)

narrowband_thresh = torch.tensor(0,dtype = torch.float, device = device)

model = Gaussian_psf(box_shape,box_size[:,1],sigma = 1e3,device = device)
PSF = model.forward()
OTF = torch.abs(torch.fft.fftshift(torch.fft.fftn(PSF)))

meshFT = Fourier3dMesh(box_size,list(box_shape),device=device, dtype = torch.float32)
ftmesh = meshFT(Verts,Faces, Faces_coeff)
image_ft=normalize_tensor(render_image_from_ftmesh(ftmesh, OTF, list(box_shape)))

In [ ]:
import napari
v = napari.view_image(labels)
v.add_image(contours)
v.add_image(image_ft.detach().cpu().numpy())